In [1]:
"""
Radar HR/RR Estimation Script (Refactored)
------------------------------------------
- Decodes IWR6843 radar binary data
- Extracts beam signals using MVDR
- Estimates HR and RR from each beam
- Matches estimates with GT using the Hungarian algorithm
- Saves results in CSV and NPY formats
"""

import os
import re
import glob
import numpy as np
import pandas as pd
import logging
from scipy.signal import butter, filtfilt, hilbert, medfilt, find_peaks, welch
from scipy.optimize import linear_sum_assignment

# --- CONFIGURATION ---
DATA_DIR = "plots/Top"
ANNOTATIONS_XLSX = "plots/Top/annotations_Top.xlsx"
OUT_CSV = "plots/Top/annotation_with_estimates_set_top.csv"
ALL_BEAMS_NPY = "plots/Top/all_beams_top.npy"
BEAM_META_CSV = "plots/Top/all_beam_meta_Settop.csv"

logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')

# --- UTILITY FUNCTIONS ---
def decode_iwr6843_data(filename, num_rx=4, num_adc_samples=256, num_chirps=128, header_bytes=0):
    with open(filename, 'rb') as f:
        f.seek(header_bytes)
        raw = np.fromfile(f, dtype=np.int16)
    raw = raw.reshape(-1, 2)
    complex_data = raw[:, 0] + 1j * raw[:, 1]
    samp_pf = num_rx * num_chirps * num_adc_samples
    num_frames = len(complex_data) // samp_pf
    adc = np.empty((num_frames, num_rx, num_chirps, num_adc_samples), dtype=complex)
    for fr in range(num_frames):
        for rx in range(num_rx):
            st = fr * samp_pf + rx * (num_chirps * num_adc_samples)
            en = st + (num_chirps * num_adc_samples)
            adc[fr, rx] = complex_data[st:en].reshape(num_chirps, num_adc_samples)
    return adc

def compute_range_profiles(adc):
    return np.fft.fft(adc, axis=-1)

def compute_range_angle_map(rp, n_angle_bins=256):
    avg_rx = rp.mean(axis=(0, 2))
    ang = np.fft.fftshift(np.fft.fft(avg_rx, n=n_angle_bins, axis=0), axes=0)
    return np.abs(ang).T

def detect_targets_physical(ra_map, num_targets=2, min_dist_m=0.7, B=3.9e9, n_angle_bins=256, R_mean=4.0):
    c = 3e8
    rng_res = c / (2 * B)
    min_rb = int(np.ceil(min_dist_m / rng_res))
    ang_res = 180.0 / n_angle_bins
    ang_sep = np.degrees(np.arcsin(min_dist_m / R_mean))
    min_ab = int(np.ceil(ang_sep / ang_res))
    R, A = ra_map.shape
    idxs = np.argsort(ra_map.flatten())[::-1]
    targets = []
    for idx in idxs:
        r, a = divmod(idx, A)
        if r < min_rb or r > R - min_rb:
            continue
        if any(abs(r - r0) < min_rb and abs(a - a0) < min_ab for r0, a0 in targets):
            continue
        targets.append((r, a))
        if len(targets) == num_targets:
            break
    return targets

def separate_with_mvdr(adc, targets, n_angle_bins=256, fc=60e9, rng_win=3, eps=1e-6):
    frames, num_rx, chirps, samples = adc.shape
    rp = np.fft.fft(adc, axis=-1)
    rng_bins = rp.shape[-1]
    angle_axis = np.linspace(-90, 90, n_angle_bins)
    def steer(theta):
        lam = 3e8 / fc
        d = lam / 2
        k = 2 * np.pi / lam
        idx = np.arange(num_rx)
        return np.exp(-1j * k * d * idx * np.sin(np.deg2rad(theta)))[:, None]
    beams = np.zeros((frames, len(targets)), dtype=complex)
    for i, (rbin, abin) in enumerate(targets):
        lo, hi = max(0, rbin - rng_win), min(rng_bins, rbin + rng_win + 1)
        gated = rp[:, :, :, lo:hi]
        X = gated.transpose(1, 0, 2, 3).reshape(num_rx, -1)
        Rcov = X @ X.conj().T / X.shape[1] + eps * np.eye(num_rx)
        a_vec = steer(angle_axis[abin])
        w = np.linalg.inv(Rcov) @ a_vec
        w /= (a_vec.conj().T @ np.linalg.inv(Rcov) @ a_vec)
        for fr in range(frames):
            Y = gated[fr].reshape(num_rx, -1)
            beams[fr, i] = (w.conj().T @ Y).sum()
    return beams

def bandpass_filter(x, low, high, fs, order=4):
    b, a = butter(order, [low / (fs / 2), high / (fs / 2)], btype='band')
    return filtfilt(b, a, x)

def estimate_rate_welch(x, fs, fmin, fmax):
    f, P = welch(x, fs=fs, nperseg=min(len(x), int(5 * fs)))
    mask = (f >= fmin) & (f <= fmax)
    return (f[mask][np.argmax(P[mask])] * 60.0) if np.any(mask) else np.nan

def estimate_hr_from_beam(z, fs):
    phase = np.unwrap(np.angle(z))
    dphi = np.gradient(phase) * fs / (2 * np.pi)
    heart = bandpass_filter(dphi, 0.8, 3.0, fs)
    return estimate_rate_welch(heart, fs, 0.8, 3.0)

def estimate_rr_from_beam_peaks(z, fs, rr_band=(0.1, 0.6), order=4, min_rr=0, max_rr=35, medfilt_kernel=5, prom_factor=0.3):
    phase = np.unwrap(np.angle(z))
    dphi = np.gradient(phase) * fs / (2 * np.pi)
    b, a = butter(order, [rr_band[0] / (fs / 2), rr_band[1] / (fs / 2)], btype='band')
    instf = filtfilt(b, a, dphi)
    env = np.abs(hilbert(instf))
    env_s = medfilt(env, kernel_size=medfilt_kernel)
    min_dist = int(fs * 60.0 / max_rr)
    prom = prom_factor * np.std(env_s)
    peaks, _ = find_peaks(env_s, distance=min_dist, prominence=prom)
    if len(peaks) < 2:
        peaks, _ = find_peaks(env_s, distance=min_dist, prominence=0.1 * np.std(env_s))
    if len(peaks) >= 2:
        ibis = np.diff(peaks) / fs
        bpm = 60.0 / ibis
        valid = bpm[(bpm >= min_rr) & (bpm <= max_rr)]
        if len(valid) > 0:
            return float(np.mean(valid)), peaks
    return estimate_rate_welch(instf, fs, rr_band[0], rr_band[1]), np.array([])

def extract_num_patients(fname):
    match = re.search(r"Man(\\d+)", fname, re.IGNORECASE)
    return int(match.group(1)) if match else 2

def assign_gt_to_est(gt_list, est_list):
    gt = pd.Series(gt_list).dropna().astype(float).values
    est = pd.Series(est_list).dropna().astype(float).values
    if len(gt) == 0 or len(est) == 0:
        return []
    cost = np.abs(gt[:, None] - est[None, :])
    gt_idx, est_idx = linear_sum_assignment(cost)
    return [(i, j, gt[i], est[j], gt[i] - est[j]) for i, j in zip(gt_idx, est_idx)]

# --- MAIN EXECUTION ---
def main():
    ann = pd.read_excel(ANNOTATIONS_XLSX)
    results_rows, all_beams, all_beams_meta = [], [], []

    for idx, row in ann.iterrows():
        fname = row['fname']
        files = glob.glob(os.path.join(DATA_DIR, f"{fname}*.bin"))
        if not files:
            logging.warning(f"No file found for {fname}")
            continue
        binfile = files[0]
        adc_samples = int(row.get('num_adc_samples', 256))
        num_chirps = int(row.get('num_chirps', 128))
        fs = float(row.get('framerate', 40.0))
        num_targets = extract_num_patients(fname)

        gt_hrs = [row[k] for k in row.keys() if re.match(r"gt_hr\\d*$", k)]
        gt_rrs = [row[k] for k in row.keys() if re.match(r"gt_rr\\d*$", k)]
        if not gt_hrs and 'gt_hr' in row: gt_hrs.append(row['gt_hr'])
        if not gt_rrs and 'gt_rr' in row: gt_rrs.append(row['gt_rr'])

        try:
            adc = decode_iwr6843_data(binfile, num_rx=4, num_adc_samples=adc_samples, num_chirps=num_chirps)
        except Exception as e:
            logging.error(f"Error decoding {binfile}: {e}")
            continue

        rp = compute_range_profiles(adc)
        ra_map = compute_range_angle_map(rp)
        targets = detect_targets_physical(ra_map, num_targets=num_targets)
        beams = separate_with_mvdr(adc, targets)
        all_beams.append(beams)

        for i in range(beams.shape[1]):
            all_beams_meta.append({"fname": fname, "beam_idx": i+1, **{col: row[col] for col in ann.columns}})

        hr_ests, rr_ests = [], []
        for i in range(beams.shape[1]):
            z = beams[:, i]
            hr_ests.append(estimate_hr_from_beam(z, fs))
            rr, _ = estimate_rr_from_beam_peaks(z, fs)
            rr_ests.append(rr)

        for (i_gt, i_est, gt_val, est_val, diff) in assign_gt_to_est(gt_hrs, hr_ests):
            results_rows.append({"fname": fname, "type": "HR", "gt_idx": i_gt, "beam_idx": i_est, "gt": gt_val, "est": est_val, "diff": diff, **{col: row[col] for col in ann.columns}})

        for (i_gt, i_est, gt_val, est_val, diff) in assign_gt_to_est(gt_rrs, rr_ests):
            results_rows.append({"fname": fname, "type": "RR", "gt_idx": i_gt, "beam_idx": i_est, "gt": gt_val, "est": est_val, "diff": diff, **{col: row[col] for col in ann.columns}})

    if all_beams:
        minlen = min(b.shape[0] for b in all_beams)
        trimmed = [b[:minlen] for b in all_beams]
        np.save(ALL_BEAMS_NPY, np.concatenate(trimmed, axis=1))
        pd.DataFrame(all_beams_meta).to_csv(BEAM_META_CSV, index=False)
        logging.info(f"Saved beams to {ALL_BEAMS_NPY} and {BEAM_META_CSV}")

    pd.DataFrame(results_rows).to_csv(OUT_CSV, index=False)
    logging.info(f"Saved results to {OUT_CSV}")
    print(pd.DataFrame(results_rows)[['fname', 'type', 'gt', 'est', 'diff']].head(20))

if __name__ == "__main__":
    main()

results_df = pd.DataFrame(results_rows)
results_df.to_csv(OUT_CSV, index=False)
logging.info(f"Saved results to {OUT_CSV}")

if not results_df.empty:
    print(results_df[['fname', 'type', 'gt', 'est', 'diff']].head(20))
else:
    logging.warning("No results to display.")



[INFO] Saved beams to plots/Top/all_beams_top.npy and plots/Top/all_beam_meta_Settop.csv
[INFO] Saved results to plots/Top/annotation_with_estimates_set_top.csv


                                                fname type     gt         est  \
0   078_TopFront_2m_45deg_Man2_CondA_CondA_ADC256_...   HR   80.0   60.000000   
1   078_TopFront_2m_45deg_Man2_CondA_CondA_ADC256_...   RR   14.0   15.742632   
2   078_TopFront_2m_45deg_Man2_CondA_CondA_ADC256_...   HR   80.0   60.000000   
3   078_TopFront_2m_45deg_Man2_CondA_CondA_ADC256_...   RR   14.0   15.742632   
4   079_TopFront_2m_45deg_Man2_CondA_CondB_ADC256_...   HR   80.0  108.000000   
5   079_TopFront_2m_45deg_Man2_CondA_CondB_ADC256_...   RR   14.0   20.676197   
6   079_TopFront_2m_45deg_Man2_CondA_CondB_ADC256_...   HR   60.0  108.000000   
7   079_TopFront_2m_45deg_Man2_CondA_CondB_ADC256_...   RR   12.0    6.153846   
8   080_TopFront_2m_45deg_Man2_CondA_CondC_ADC256_...   HR   80.0   84.000000   
9   080_TopFront_2m_45deg_Man2_CondA_CondC_ADC256_...   RR   14.0   16.216216   
10  080_TopFront_2m_45deg_Man2_CondA_CondC_ADC256_...   HR  110.0   84.000000   
11  080_TopFront_2m_45deg_Ma

NameError: name 'results_rows' is not defined